# A Classic Neural Networks Tutorial

For this tutorial, we'll be using Keras. Keras is like a higher-level abstraction of Tensorflow -- a popular ML library -- and will allow us to do some pretty cool stuff without knowing a lot of linear algebra/calculus. :)

In [1]:
# fashion dataset import
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

## The dataset - Fashion MNIST

We'll be using the MNIST (Modified National Institute of Standards and Technology database). They have a large database of handwritten digit images, but we'll be analyzing [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) for the following reasons:

**1.** The MNIST digit dataset is too easy, and even classic ML algorithms can achieve 97% accuracy on this dataset.

**2.** MNIST is overused.

**3.** MNIST is not very representative of stuff you might actually do; i.e. bad ideas might work well on the digit dataset, but not on most other datasets.

![title](img/fashion-dataset-ov.png)

### The Goal

We want to turn each of these 28x28 pixel, grayscale images into one of 10 classifications:

Label | Description
--- | --- 
0 | T-shirt/top
1 | Trouser
2 | Pullover
3 | Dress
4 | Coat
5 | Sandal
6 | Shirt
7 | Sneaker
8 | Bag
9 | Ankle Boot

The dataset is already presplit into 60,000 training images and 10,000 testing images. Let's get started by splitting the data into training, testing, and validation data!

## Intro to Perceptrons

![title](img/perceptron.png)

Notice the structure is a series of connected, feed-forward neurons (directed forward toward the output layer). We feed in data into the input layer, and the hidden layer neurons fire accordingly and communicate with the output layer.

Also notice the input layer is flat, whereas our data is 2D. How can we solve this?

### Flattening the data

In [9]:
# Load in data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [10]:
x_train.shape

(60000, 28, 28)

In [11]:
# Flatten data from 2D to 1D:
# reshape(__, -1) means that we're telling numpy that there are
# two dimensions, and to infer the second dimension
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
# note 28 * 28 = 784!
x_train.shape

(60000, 784)

In [14]:
# Category
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

### Creating the Model

We'll be using the ```Sequential``` model with ```Dense``` layers. If you're curious, a ```Dense``` layer is just a regular NN layer that does: ```output = activation(dot(input, kernel) + bias)```.

This means that it takes the dot product of your input vector and a weight matrix, and adds a bias vector.

In [16]:
# Literally the simplest neural net model that Keras has
# Sequential is a linear stack of layers
model = Sequential()

In [18]:
# add layers to model
# also note the activation functions we're using
model.add(Dense(10, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Train
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 3s 48us/sample - loss: 0.4255 - accuracy: 0.8604 - val_loss: 0.4593 - val_accuracy: 0.8530
Epoch 2/10
54000/54000 [==============================] - 3s 47us/sample - loss: 0.4157 - accuracy: 0.8621 - val_loss: 0.4575 - val_accuracy: 0.8520
Epoch 3/10
54000/54000 [==============================] - 3s 48us/sample - loss: 0.4061 - accuracy: 0.8649 - val_loss: 0.4418 - val_accuracy: 0.8543
Epoch 4/10
54000/54000 [==============================] - 3s 50us/sample - loss: 0.3980 - accuracy: 0.8658 - val_loss: 0.4438 - val_accuracy: 0.8555
Epoch 5/10
54000/54000 [==============================] - 3s 50us/sample - loss: 0.3919 - accuracy: 0.8674 - val_loss: 0.4475 - val_accuracy: 0.8505
Epoch 6/10
54000/54000 [==============================] - 3s 49us/sample - loss: 0.3859 - accuracy: 0.8692 - val_loss: 0.4428 - val_accuracy: 0.8548
Epoch 7/10
54000/54000 [==============================] -

In [21]:
# Widen network
model2 = Sequential()
model2.add(Dense(50, input_dim=784, activation='relu'))
model2.add(Dense(10, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 3s 57us/sample - loss: 0.5390 - accuracy: 0.8142 - val_loss: 0.4136 - val_accuracy: 0.8540
Epoch 2/10
54000/54000 [==============================] - 3s 48us/sample - loss: 0.4080 - accuracy: 0.8552 - val_loss: 0.3948 - val_accuracy: 0.8585
Epoch 3/10
54000/54000 [==============================] - 2s 46us/sample - loss: 0.3689 - accuracy: 0.8677 - val_loss: 0.3760 - val_accuracy: 0.8652
Epoch 4/10
54000/54000 [==============================] - 3s 48us/sample - loss: 0.3441 - accuracy: 0.8753 - val_loss: 0.3756 - val_accuracy: 0.8690
Epoch 5/10
54000/54000 [==============================] - 3s 46us/sample - loss: 0.3293 - accuracy: 0.8800 - val_loss: 0.3668 - val_accuracy: 0.8713
Epoch 6/10
54000/54000 [==============================] - 2s 46us/sample - loss: 0.3169 - accuracy: 0.8834 - val_loss: 0.3409 - val_accuracy: 0.8782
Epoch 7/10
54000/54000 [==============================] -

In [22]:
# Deeper
model3 = Sequential()
model3.add(Dense(50, input_dim=784, activation='relu'))
model3.add(Dense(50, activation='relu'))
model3.add(Dense(10, activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 3s 60us/sample - loss: 0.5286 - accuracy: 0.8118 - val_loss: 0.4170 - val_accuracy: 0.8490
Epoch 2/10
54000/54000 [==============================] - 3s 53us/sample - loss: 0.3957 - accuracy: 0.8566 - val_loss: 0.3638 - val_accuracy: 0.8657
Epoch 3/10
54000/54000 [==============================] - 3s 51us/sample - loss: 0.3600 - accuracy: 0.8689 - val_loss: 0.3506 - val_accuracy: 0.8722
Epoch 4/10
54000/54000 [==============================] - 3s 51us/sample - loss: 0.3351 - accuracy: 0.8761 - val_loss: 0.3542 - val_accuracy: 0.8738
Epoch 5/10
54000/54000 [==============================] - 3s 54us/sample - loss: 0.3157 - accuracy: 0.8832 - val_loss: 0.3669 - val_accuracy: 0.8697
Epoch 6/10
54000/54000 [==============================] - 3s 52us/sample - loss: 0.3016 - accuracy: 0.8875 - val_loss: 0.3420 - val_accuracy: 0.8767
Epoch 7/10
54000/54000 [==============================] -

In [23]:
_, test_acc = model3.evaluate(x_test, y_test)
print(test_acc)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.8773


### Convolutional Neural Nets

In [25]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
import numpy as np
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train[:,:,:,np.newaxis] / 255.0
x_test = x_test[:,:,:,np.newaxis] / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [27]:
model4 = Sequential()
model4.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28, 1))) 
model4.add(MaxPooling2D(pool_size=2))
model4.add(Flatten())
model4.add(Dense(10, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                125450    
Total params: 125,770
Trainable params: 125,770
Non-trainable params: 0
_________________________________________________________________


In [29]:
model4.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 22s 406us/sample - loss: 0.4331 - accuracy: 0.8498 - val_loss: 0.3306 - val_accuracy: 0.8822
Epoch 2/10
54000/54000 [==============================] - 23s 427us/sample - loss: 0.3086 - accuracy: 0.8912 - val_loss: 0.3057 - val_accuracy: 0.8918
Epoch 3/10
54000/54000 [==============================] - 24s 445us/sample - loss: 0.2773 - accuracy: 0.9017 - val_loss: 0.2832 - val_accuracy: 0.9003
Epoch 4/10
54000/54000 [==============================] - 24s 454us/sample - loss: 0.2558 - accuracy: 0.9094 - val_loss: 0.2858 - val_accuracy: 0.8960
Epoch 5/10
54000/54000 [==============================] - 21s 393us/sample - loss: 0.2412 - accuracy: 0.9144 - val_loss: 0.2743 - val_accuracy: 0.9022
Epoch 6/10
54000/54000 [==============================] - 19s 360us/sample - loss: 0.2256 - accuracy: 0.9195 - val_loss: 0.2793 - val_accuracy: 0.9020
Epoch 7/10
54000/54000 [=====================

In [30]:
_, test_acc = model4.evaluate(x_test, y_test)
print(test_acc)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.8988
